In [1]:
from pvtpy.pvt import PVT
import pvtpy.black_oil as bk
from pvtpy.fluids import Gas, InitialConditions
from pvtpy.pvt import PVT
from pvtpy.compositional import Chromatography
from pvtpy.units import Pressure, Temperature
import numpy as np
import pandas as pd
import seaborn as sns

# Create a chromatography composition

In [2]:
d = {
'comp':['carbon-dioxide','nitrogen','methane','ethane','propane','isobutane','n-butane'],
'mole_fraction':[0.02,0.01,0.85,0.04,0.03,0.03,0.02]
}
cc = pd.DataFrame(d)
chr = Chromatography()
chr.from_df(cc,name='comp')
print(chr.df())

                acentric_factor        cas  critical_pressure  \
carbon-dioxide           0.2276   124-38-9             1071.0   
nitrogen                 0.0403  7727-37-9              493.1   
methane                  0.0115    74-82-8              666.4   
ethane                   0.0995    74-84-0              706.5   
propane                  0.1523    74-98-6              616.0   
isobutane                0.1770    75-28-5              527.9   

                critical_temperature formula    id  \
carbon-dioxide                 87.91     CO2  50.0   
nitrogen                     -232.51      N2  56.0   
methane                      -116.67     CH4   1.0   
ethane                         89.92    C2H6   2.0   
propane                       206.06    C3H8   3.0   
isobutane                     274.46   C4H10   4.0   

                                      iupac                    iupac_key  \
carbon-dioxide                1S/CO2/c2-1-3  CURLTUGMZLYLDI-UHFFFAOYSA-N   
nitrogen     

## Estimate some properties

### Apparent Molecular Weigt

The apparent Molecular Weight (ma) is calculated by summing the product of molar fraction and molecular weight of each component in the chromatography

In [3]:
chr.apparent_molecular_weight()

19.4549581632653

### Gas specific gravity

The Gas specific gravity is calculated by diving the **ma** by the specific gravity of the *air*

In [4]:
chr.gas_sg()

0.67178722939452

### Pseudo critical properties

The Pseudo critical properties are calulated by summing the product of mole fraction and critical properties (pressure and temperature). By default it corrects the properties by Non-hydrocarbon components with the **wichert-aziz** correlation. 

In [5]:
chr.get_pseudo_critical_properties()

CriticalProperties(critical_pressure=Pressure(value=662.7409312376316, unit=<PressureUnits.psi: 'psi'>), critical_temperature=Temperature(value=372.9010899529697, unit=<TemperatureUnits.farenheit: 'farenheit'>))

In [6]:
chr.get_pseudo_critical_properties(correct=False)

CriticalProperties(critical_pressure=Pressure(value=668.742857142857, unit=<PressureUnits.psi: 'psi'>), critical_temperature=Temperature(value=376.27816326530603, unit=<TemperatureUnits.rankine: 'rankine'>))

In [7]:
chr.get_pseudo_critical_properties(correct_method='carr_kobayashi_burrows')

CriticalProperties(critical_pressure=Pressure(value=675.9877551020408, unit=<PressureUnits.psi: 'psi'>), critical_temperature=Temperature(value=372.09448979591826, unit=<TemperatureUnits.farenheit: 'farenheit'>))

### Get the compressibility factor of gas

Estimate the compressibility factor by estimating the critical properties and applying the default correlation method **papay**

In [8]:
chr.get_z(pressure=3000, temperature=180)

,z
pressure,
3000,-0.195114


In [9]:
p_range = np.linspace(1000,5000,10)
chr.get_z(pressure=p_range, temperature=180)

,z
pressure,
1000.000000,0.498960
1444.444444,0.309231
1888.888889,0.139782
2333.333333,-0.009386
2777.777778,-0.138275
3222.222222,-0.246882
3666.666667,-0.335210
4111.111111,-0.403258
4555.555556,-0.451025


### Get the gas density in lb/ft3

Estimate the gas density by estimating the **ma**, the **z** factor and finnaly applying the gas equation of state for **real gases**

In [10]:
chr.get_rhog(pressure=3000,temperature=180)

,rhog
pressure,
3000,-43.559656


In [11]:
chr.get_rhog(pressure=3000,temperature=180, rhog_method='ideal_gas')

,rhog
pressure,
3000,8.499079


In [12]:
chr.get_rhog(pressure=np.linspace(1000,5000,10),temperature=180,rhog_method='real_gas')

,rhog
pressure,
1000.000000,5.677863
1444.444444,13.233311
1888.888889,38.282935
2333.333333,-704.258905
2777.777778,-56.912269
3222.222222,-36.975650
3666.666667,-30.988807
4111.111111,-28.882001
4555.555556,-28.614858


### Estimate the Specific volume of Gas. 

Get the specific volume by estimate the inverse of the density

In [13]:
chr.get_sv(pressure=3000,temperature=180, rhog_method='ideal_gas')

pressure
3000    0.11766
Name: sv, dtype: float64

In [14]:
chr.get_sv(pressure=3000,temperature=180, rhog_method='real_gas')

pressure
3000   -0.022957
Name: sv, dtype: float64

# Create the Gas object

In [15]:
init = InitialConditions(pressure=Pressure(value=3600), temperature=Temperature(value=160))

In [16]:
gas1 = Gas(
    initial_conditions=init,sg=0.6,
    chromatography=chr
)


In [17]:
gas1.pseudo_critical_properties()

CriticalProperties(critical_pressure=Pressure(value=662.7409312376316, unit=<PressureUnits.psi: 'psi'>), critical_temperature=Temperature(value=372.9010899529697, unit=<TemperatureUnits.farenheit: 'farenheit'>))

In [18]:
gas1.pvt_from_correlations()
gas1.pvt.df()

/Users/scuervo91/Documents/dev/pvtpy/pvtpy/black_oil/correlations.py:1488: RuntimeWarning: invalid value encountered in power
  mug = 1e-4 * k * np.exp(x*np.power(rhog/62.4,y))


,z,rhog,bg,mug,cg
pressure,,,,,
20.000000,0.988358,0.059177,0.154115,1.261362e-02,0.050000
282.105263,0.839758,0.982417,0.009283,1.286292e-02,0.003545
544.210526,0.698536,2.278331,0.004003,1.341415e-02,0.001838
806.315789,0.564693,4.175720,0.002184,1.450051e-02,0.001240
1068.421053,0.438229,7.129846,0.001279,1.679766e-02,0.000936
1330.526316,0.319143,12.192040,0.000748,2.273917e-02,0.000752
1592.631579,0.207437,22.452720,0.000406,4.811116e-02,0.000628
1854.736842,0.103108,52.605092,0.000173,8.423217e-01,0.000539
2116.842105,0.006159,1005.181931,0.000009,1.060569e+94,0.000472
